In [1]:
import glob
import re
import matplotlib as plt
import numpy as np
import scipy as sp
import xarray as xr
import intake
import intake_esm
import pandas as pd

In [2]:
from dask_jobqueue import PBSCluster
from distributed import Client

In [3]:
######## File paths ################
lustre_scratch  = "/lustre/desc1/scratch/harshah"
gdex_data       = "/gdex/data/"
# gdex_url      = 'https://data.gdex.ucar.edu/'
eol_data        = "/gdex/data/special_projects/pythia_2025/eol-cookbook/m2hats_iss2_data/prof449Mhz_30min_winds/"
#########
era5_catalog      =  gdex_data + 'special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json'
print(era5_catalog)

/gdex/data/special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json


In [4]:
cluster = PBSCluster(
        job_name = 'dask-eol-25',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = lustre_scratch + '/dask/spill',
        log_directory = lustre_scratch + '/dask/logs/',
        resource_spec = 'select=1:ncpus=1:mem=4GB',
        queue = 'casper',
        walltime = '3:00:00',
        #interface = 'ib0'
        interface = 'ext')

In [5]:
client = Client(cluster)

In [6]:
# Scale the cluster and display cluster dashboard URL
n_workers =5
cluster.scale(n_workers)
client.wait_for_workers(n_workers = n_workers)
cluster

PBSCluster(16b54020, 'tcp://128.117.208.96:38099', workers=5, threads=5, memory=20.00 GiB)

### Load data

In [7]:
# %%time
# prof449_wind = xr.open_mfdataset(eol_data + '*.nc',concat_dim = 'time',combine='nested')

In [8]:
prof449Mhz_wind_test = xr.open_dataset(eol_data + 'prof449.20230926.winds.30.nc')
prof449Mhz_wind_test

<xarray.Dataset> Size: 122kB
Dimensions:       (time: 48, height: 55)
Coordinates:
  * time          (time) datetime64[ns] 384B 2023-09-26T00:15:00 ... 2023-09-...
    height        (time, height) float32 11kB ...
Data variables: (12/18)
    base_time     datetime64[ns] 8B ...
    lat           float32 4B ...
    lon           float32 4B ...
    alt           float32 4B ...
    frequency     float32 4B ...
    x_spacing     float32 4B ...
    ...            ...
    snrw          (time, height) float32 11kB ...
    specWid       (time, height) float32 11kB ...
    u_dispersion  (time, height) float32 11kB ...
    v_dispersion  (time, height) float32 11kB ...
    w_dispersion  (time, height) float32 11kB ...
    cons_npoints  (time, height) int16 5kB ...
Attributes: (12/16)
    description:        Modular Profiler consensus winds
    Processing-Method:  30 minute consensus averages
    Processing-Ops:       read_raw_winds mapr_nc szl yyyy correct_szl correct...
    history:            write_cons_5.pro
    author:             Bill Brown, NCAR/EOL
    date:               Mon Jul  1 10:04:28 2024   1719849868
    ...                 ...
    project:            M2HATS
    site:               Tonopah Airport, NV
    missing_value:      -999.0
    zebra_platform:     MAPR/449 winds
    first_file:         /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...
    last_file:          /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...

In [9]:
# %%time
# prof449Mhz_wind_test.to_zarr(eol_data + 'prof449.20230926.winds.30.zarr')

In [10]:
prof449Mhz_wind_test_zarr = xr.open_zarr(eol_data + 'prof449.20230926.winds.30.zarr')
prof449Mhz_wind_test_zarr

<xarray.Dataset> Size: 122kB
Dimensions:       (time: 48, height: 55)
Coordinates:
    height        (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
  * time          (time) datetime64[ns] 384B 2023-09-26T00:15:00 ... 2023-09-...
Data variables: (12/18)
    alt           float32 4B ...
    base_time     datetime64[ns] 8B ...
    cons_npoints  (time, height) int16 5kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    frequency     float32 4B ...
    lat           float32 4B ...
    lon           float32 4B ...
    ...            ...
    v_dispersion  (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    w_dispersion  (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    wdir          (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    wspd          (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    wvert         (time, height) float32 11kB dask.array<chunksize=(48, 55), meta=np.ndarray>
    x_spacing     float32 4B ...
Attributes: (12/16)
    File-End-Time:      1695772799.0000055
    File-Start-Time:    1695686400.0
    Processing-Method:  30 minute consensus averages
    Processing-Ops:       read_raw_winds mapr_nc szl yyyy correct_szl correct...
    author:             Bill Brown, NCAR/EOL
    beg_dt:             [2023.0, 9.0, 26.0, 0.0, 0.0, 0.0]
    ...                 ...
    history:            write_cons_5.pro
    last_file:          /scr/isf_apg/raw/m2hats/iss1/modular_prof/reprocessed...
    missing_value:      -999.0
    project:            M2HATS
    site:               Tonopah Airport, NV
    zebra_platform:     MAPR/449 winds

### Load ERA5 data

In [11]:
%%time
era5_cat = intake.open_esm_datastore(era5_catalog)
era5_cat.df

/glade/u/home/harshah/venvs/osdf/lib/python3.10/site-packages/intake_esm/cat.py:250: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


CPU times: user 1.96 s, sys: 197 ms, total: 2.16 s
Wall time: 2.23 s


,era_id,datatype,level_type,step_type,table_code,param_code,variable,long_name,units,year,month,format,frequency,path
0,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.an.p...
1,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.an.p...
2,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.an.p...
3,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.an.p...
4,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.an.p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785063,e5,fc,sfc,minmax,128,202,MN2T,Minimum temperature at 2 metres since previous...,K,2024,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.fc.s...
785064,e5,fc,sfc,minmax,228,226,MXTPR,Maximum total precipitation rate since previou...,kg m**-2 s**-1,2024,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.fc.s...
785065,e5,fc,sfc,minmax,228,226,MXTPR,Maximum total precipitation rate since previou...,kg m**-2 s**-1,2024,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.fc.s...
785066,e5,fc,sfc,minmax,228,227,MNTPR,Minimum total precipitation rate since previou...,kg m**-2 s**-1,2024,1,nc,hourly,https://data.rda.ucar.edu/d633000/e5.oper.fc.s...


In [12]:
era5_cat.df['variable'].unique()

array(['PV', 'CRWC', 'CSWC', 'Z', 'T', 'U', 'V', 'Q', 'W', 'VO', 'D', 'R',
       'O3', 'CLWC', 'CIWC', 'CC', 'ALUVP', 'ALUVD', 'ALNIP', 'ALNID',
       'CI', 'ASN', 'RSN', 'SSTK', 'ISTL1', 'ISTL2', 'ISTL3', 'ISTL4',
       'SWVL1', 'SWVL2', 'SWVL3', 'SWVL4', 'CAPE', 'LAILV', 'LAIHV',
       'TCLW', 'TCIW', 'SP', 'TCW', 'TCWV', 'STL1', 'SD', 'CHNK', 'MSL',
       'BLH', 'TCC', 'VAR_10U', 'VAR_10V', 'VAR_2T', 'VAR_2D', 'STL2',
       'STL3', 'LCC', 'MCC', 'HCC', 'SRC', 'TCO3', 'IEWS', 'INSS', 'ISHF',
       'IE', 'SKT', 'STL4', 'TSN', 'FAL', 'FSR', 'FLSR', 'LBLT', 'LTLT',
       'LSHF', 'LICT', 'LICD', 'TCRW', 'TCSW', 'U10N', 'V10N', 'VAR_100U',
       'VAR_100V', 'LMLT', 'LMLD', 'VIMA', 'VIT', 'VIKE', 'VITHE',
       'VIPIE', 'VIPILE', 'VITOE', 'VIEC', 'VIMAE', 'VIMAN', 'VIKEE',
       'VIKEN', 'VITHEE', 'VITHEN', 'VIWVE', 'VIWVN', 'VIGE', 'VIGN',
       'VITOEE', 'VITOEN', 'VIOZE', 'VIOZN', 'VILWD', 'VIIWD', 'VIMAD',
       'VIKED', 'VITHED', 'VIWVD', 'VIGD', 'VITOED', 'VIOZD', 'VILWE

In [13]:
cat_subset = era5_cat.search(variable=['T'],frequency = 'hourly',year=2023,month=[7,8,9])
cat_subset

,unique
era_id,1
datatype,1
level_type,0
step_type,1
table_code,1
param_code,1
variable,1
long_name,1
units,1
year,1


In [14]:
# Define the xarray_open_kwargs with a compatible engine, for example, 'scipy'
xarray_open_kwargs = {
    'engine': 'h5netcdf',
    'chunks': {},  # Specify any chunking if needed
    'backend_kwargs': {}  # Any additional backend arguments if required
}

In [15]:
%%time
dsets = cat_subset.to_dataset_dict(xarray_open_kwargs=xarray_open_kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'datatype.step_type'


CPU times: user 2.17 s, sys: 97.6 ms, total: 2.27 s
Wall time: 32.2 s


In [16]:
dset = dsets['an.pl']
dset 

<xarray.Dataset> Size: 339GB
Dimensions:    (time: 2208, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 18kB 2023-07-01 ... 2023-09-30T23:00:00
    utc_date   (time) int32 9kB dask.array<chunksize=(24,), meta=np.ndarray>
Data variables:
    T          (time, level, latitude, longitude) float32 339GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
Attributes: (12/20)
    DATA_SOURCE:                     ECMWF: https://cds.climate.copernicus.eu...
    NETCDF_CONVERSION:               CISL RDA: Conversion from ECMWF GRIB 1 d...
    NETCDF_VERSION:                  4.8.1
    Conventions:                     CF-1.6
    NETCDF_COMPRESSION:              NCO: Precision-preserving compression to...
    NCO:                             netCDF Operators version 5.0.3 (Homepage...
    ...                              ...
    intake_esm_attrs:units:          K
    intake_esm_attrs:year:           2023
    intake_esm_attrs:format:         nc
    intake_esm_attrs:frequency:      hourly
    intake_esm_attrs:_data_format_:  netcdf
    intake_esm_dataset_key:          an.pl

In [18]:
temps = dset.T

In [19]:
temps

<xarray.DataArray 'T' (time: 2208, level: 37, latitude: 721, longitude: 1440)> Size: 339GB
dask.array<concatenate, shape=(2208, 37, 721, 1440), dtype=float32, chunksize=(1, 37, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) float64 296B 1.0 2.0 3.0 5.0 ... 925.0 950.0 975.0 1e+03
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 18kB 2023-07-01 ... 2023-09-30T23:00:00
    utc_date   (time) int32 9kB dask.array<chunksize=(24,), meta=np.ndarray>
Attributes:
    long_name:                                          Temperature
    short_name:                                         t
    units:                                              K
    original_format:                                    WMO GRIB 1 with ECMWF...
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    130
    grid_specification:                                 0.25 degree x 0.25 de...
    rda_dataset:                                        ds633.0
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric pres...
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7